In [79]:
def verify_wallet_stats(userid, address, quantity):
    with open(f"""{puurrtybot.PATH}/puurrtybot/databases/verify_wallet/{address}.json""", "w") as f:
            json.dump({"userid":f"{userid}","quantity":f"{quantity}","verified":"False"}, f)



In [43]:
quantity = str(int(float(j["quantity"])*10**6))
quantity,j

('1379280',
 {'userid': '642352900357750787',
  'quantity': '1.37928',
  'verified': 'False',
  'time': '11111111'})

In [ ]:
def get_quantities_by_address(address: str, time_limit = 70*60):
    quantities = []
    for tx in get_tx_list_by_address(address):
        if tx['block_time'] - int(datetime.datetime.now().timestamp()) + time_limit > 0:
            quantities += get_quantities_by_tx_hash(tx['tx_hash'], address)
    return quantities

In [44]:
import puurrtybot, json
import puurrtybot.blockchain.blockchain_queries as bq

address = "addr1qyqggc5f3dgyx6ulwl4uqf8gucxvdahgpatjx27hutsusg79nee6glazchetycv3uewpraf7tfe60t3kud5l0cdkl5wqyj5xhy"

def check_wallet_for_tx(address, quantity):
    try:
        with open(f"""{puurrtybot.PATH}/puurrtybot/databases/verify_wallet/{address}.json""", 'r') as f:
            j = json.load(f)
    except FileNotFoundError:
        print('x')

    counter=0 
    for tx in bq.get_tx_list_by_address(address):
        counter+=1
        if counter>100:
            break;
        if quantity in bq.get_quantities_by_tx_hash(tx_hash=tx['tx_hash'], address=address):
            return True
    return False

check_wallet_for_tx(address, quantity)

True

In [65]:
userid = 642352900357750787
with open(f"""{puurrtybot.PATH}/puurrtybot/databases/users/{userid}.json""", 'r') as f:
    j = json.load(f)
j

{'userid': '642352900357750787',
 'verified_wallets': 'addr1qyqggc5f3dgyx6ulwl4uqf8gucxvdahgpatjx27hutsusg79nee6glazchetycv3uewpraf7tfe60t3kud5l0cdkl5wqyj5xhy'}

In [78]:
def wallet_verify_status(userid, address):
    with open(f"""{puurrtybot.PATH}/puurrtybot/databases/users/{userid}.json""", 'r') as f:
        j = json.load(f)
    if address in j['verified_wallets'].split(';'):
        return True
    else:
        return False
    
wallet_verify_status(userid, address)

True

In [71]:
userid = 642352900357750787
def add_verified_wallet(userid):
    with open(f"""{puurrtybot.PATH}/puurrtybot/databases/users/{userid}.json""", 'r') as f:
        j = json.load(f)

    j['verified_wallets']=f"""{j['verified_wallets']};{address}""".strip(';')
    j['verified_wallets']=';'.join(set(j['verified_wallets'].split(';')))
    with open(f"""{puurrtybot.PATH}/puurrtybot/databases/users/{userid}.json""", "w") as f:
        json.dump(j, f)
        
add_verified_wallet(userid)

In [57]:
with open(f"""{puurrtybot.PATH}/puurrtybot/databases/users/{userid}.json""", "w") as f:
        json.dump(j, f)

In [31]:
q = str(int(float(j["quantity"])*10**6))
q

'1379280'

In [32]:
import puurrtybot.blockchain.blockchain_queries as bq

for tx in bq.get_tx_list_by_address(address):
    if q in bq.get_quantities_by_tx_hash(tx_hash=tx['tx_hash'], address=address):
        print('check')
        break;

check


In [1]:
import puurrtybot.blockchain.meta as meta
meta.PREFIXES

['Mr.',
 "Ol' ",
 'Sir',
 "Lil' Miss ",
 'Madam ',
 'Mrs. ',
 'Prince ',
 'Lady ',
 'Princess ',
 'King ',
 'Queen ',
 'Lord ',
 'Dr. ',
 'Chairman ',
 'Professor ',
 'Chairwoman ']

In [4]:
import puurrtybot.blockchain as meta

In [50]:
    prefixes = ['Mr.',
            """Ol' """,
            'Sir',
            """Lil' Miss """,
            'Madam ',
            'Mrs. ',
            'Prince ',
            'Lady ',
            'Princess ',
            'King ',
            'Queen ',
            'Lord ',
            'Dr. ',
            'Chairman ',
            'Professor ',
            'Chairwoman ']

    suffixes = [' Esq.',
            ' Ph.D.',
            ' the Third',
            ' the Second',
            ' the First',
            ' the Fourth',
            ' v2',
            ' v3',
            ' mk IV']

In [48]:
def name_has_prefix(name):
    for prefix in prefixes:
        if prefix in name:
            return 1
    return 0

name_has_prefix(asset['onchain_metadata']['name'])

False

In [24]:
def get_addr_by_asset(asset):
    return  requests.get(f'https://cardano-mainnet.blockfrost.io/api/v0/assets/{asset}/addresses', headers=headers).json()[0]['address']


In [44]:
asset['onchain_metadata']['name']

'Wanda Boutlier'

In [52]:
import os

def name_has_prefix(name):
    for prefix in prefixes:
        if prefix in name:
            return prefix
    return None


def name_has_suffix(name):
    for suffix in suffixes:
        if suffix in name:
            return suffix
    return None

def name_has_lastname(name):
    return 0

def name_has_firstname(name):
    return 0



asset_dir = f"""{puurrtybot.PATH}/puurrtybot/databases/assets/"""
for asset in tqdm.tqdm(os.listdir(asset_dir)):
    with open(f"""{asset_dir}{asset}""", 'r') as openfile:
        asset = json.load(openfile)
        asset['onchain_metadata']['prefix'] = name_has_prefix(asset['onchain_metadata']['name'])
        asset['onchain_metadata']['suffix'] = name_has_suffix(asset['onchain_metadata']['name'])
        

100%|████████████████████████████████████| 7000/7000 [00:00<00:00, 15453.28it/s]


In [3]:
import puurrtybot, requests, json, tqdm
import puurrtybot.blockchain.blockchain_queries as blockfrost
headers = {'project_id': puurrtybot.BLOCKFROST_TOKEN}

def get_assets_by_policy(policy):
    page=1
    assets=[]
    while True:
        r = requests.get(f'https://cardano-mainnet.blockfrost.io/api/v0/assets/policy/{policy}?page={page}', headers=headers)
        if len(r.json())==0:
            break;
        else:
            assets+=r.json()
            page+=1

    return [asset['asset'] for asset in assets if asset['quantity']=='1']

def get_meta_by_asset(asset):
    return requests.get(f'https://cardano-mainnet.blockfrost.io/api/v0/assets/{asset}', headers=headers).json()

assets = []
for asset in tqdm.tqdm(blockfrost.get_assets_by_policy(puurrtybot.POLICY)):
    assets.append(blockfrost.get_meta_by_asset(asset))

assets = pd.DataFrame(assets)

100%|███████████████████████████████████████| 7000/7000 [13:44<00:00,  8.49it/s]


NameError: name 'pd' is not defined

In [1]:
import puurrtybot.functions as pf
pf.get_random_quantity()

'2.771528'

In [9]:
tx

{'tx_hash': '7a76b4938feec7b64781565a44c7a3190af4421d04ffab5c105fe8c4c19bc885',
 'tx_index': 13,
 'block_height': 7014507,
 'block_time': 1647657846}

In [8]:
import puurrtybot.blockchain.blockchain_queries as bq
address = "addr1qyqggc5f3dgyx6ulwl4uqf8gucxvdahgpatjx27hutsusg79nee6glazchetycv3uewpraf7tfe60t3kud5l0cdkl5wqyj5xhy"

for tx in bq.get_tx_list_by_address(address):
    print(tx['tx_hash'],bq.get_quantities_by_tx_hash(tx_hash=tx['tx_hash'], address=address))
    
    

883d35e57d023e09523f129af4abb00fd6db2b43629f9a1fe695859a07d97e13 [0]
bfa14c067e71565a704fd6b3f007cf57df333af85914454800a69ce05e081510 ['5000000', '1379280', '16447475']
1ecef7cda549513e589effd370072485dcbefcde5f818cb984b5ce07e7d45242 [0]
9c82987d4579be5da3bdcecccb09d1b58a2ae86925844385fdc2bb04970a6c01 [0]
033bfbcc62e557a5970f36f36e1c13e21bb4eed22eca8b677c394fb0618ffc1c [0]
6ef24315ca925bcb5d1c891ebe08b7768ff952f05cd7fa81b0ec5ef2a995f9b5 [0]
4cc2a9ae2c0323973902b68cf88941714fa3ee989ec9a9b217a35cdcda1344d6 [0]
a80c1bb707f20ee261daba783e428f322c504e1d5ba09cb9f5c2f727385b1457 [0]
328079ed3a361516e45e963f93ce29af61585e2b8a19ff30f3cbe52ed549b6cd [0]
f7a386575af30c3120b5f93a4e91cf0d10ab104c3749e9054bff6532fa6fd3d5 [0]
1474fe4bc92c38e89e71e80689ae709bef5be4ba8c8c8c0d3237b19c147ee594 [0]


KeyboardInterrupt: 

In [5]:
tx['tx_hash']

'678ed21b46f87b197125c86d47864697a8dec6f888b55df22cf8f6c7ac552c43'

In [10]:
a = [asset for asset in assets if 'f96584c4fcd13cd1702c9be683400072dd1aac853431c99037a3ab1e50757272426f74'==asset['asset']][0]

In [13]:
for asset in tqdm.tqdm(blockfrost.get_assets_by_policy(puurrtybot.POLICY)):
    asset = blockfrost.get_meta_by_asset(asset)
    with open(f"""{puurrtybot.PATH}/puurrtybot/databases/assets/{asset['asset']}.json""", 'w') as f:
        json.dump(asset, f)

{'asset': 'f96584c4fcd13cd1702c9be683400072dd1aac853431c99037a3ab1e50757272426f74',
 'policy_id': 'f96584c4fcd13cd1702c9be683400072dd1aac853431c99037a3ab1e',
 'asset_name': '50757272426f74',
 'fingerprint': 'asset12xf688slvfmtspjgnsq33wjjfp67egfy7yhwjw',
 'quantity': '1',
 'initial_mint_tx_hash': 'c3e4b98a8decfa2e6c7a56fec5d1f201ad59e80462c590e5cc81ebce4a5d6c6c',
 'mint_or_burn_count': 3,
 'onchain_metadata': {'name': 'PurrBot',
  'image': 'ipfs://QmXdUjTyPEvPAnRk5Nc1TCje7JhLvrx1F45U3oS2YLWQ74',
  'unique': 'Yes',
  'mediaType': 'image/png',
  'collection': 'Puurrty Cats Society'},
 'metadata': None}

In [15]:
f"""{a['asset']}.json"""

'f96584c4fcd13cd1702c9be683400072dd1aac853431c99037a3ab1e50757272426f74.json'

In [16]:
import json
with open(f"""{a['asset']}.json""", 'w') as f:
    json.dump(a, f)